In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

In [2]:
# 生データを引数として受け取り、そのデータの問題サイズの値に合わせた追加の初期値を追加し、引数として渡されたDFにデータが付与されたDFを返す関数
# 引数：生データDF
# 返り値：生データDFにベンチマークごとの初期値が付与されたDF


def addInitDataToRawDF(rawDF):
    # ベンチマーク名を引数から取得
    benchmarkNames = sorted(list(set(rawDF["benchmarkName"])))
    # 問題サイズごとにrawDFを分割
    benchmarkClasses = sorted(list(set(rawDF["benchmarkClass"])))
    # ベンチマーク名・問題サイズに合わせた値を格納
    for benchmarkName in benchmarkNames:
        for benchmarkClass in benchmarkClasses:
            # ベンチマーク名と問題サイズを満たす行を抽出
            extractedBools = ((rawDF['benchmarkName'] == benchmarkName) & (rawDF['benchmarkClass'] == benchmarkClass))
            # 抽出された行に値を格納
            print(rawDF[extractedBools])
            print(f"benchmarkName={benchmarkName}, benchmarkClass={benchmarkClass}")
            if len(rawDF[extractedBools]) > 0:
                dictAboutInitVars = retDictAboutInitVars(benchmarkName=benchmarkName, benchmarkClass=benchmarkClass)
                for columnName in dictAboutInitVars.keys():
                    rawDF[extractedBools][columnName] = dictAboutInitVars[columnName]
    # 問題サイズごとに分割された、修正済みDFを結合
    print(rawDF)
    return rawDF

# ベンチマーク名・問題サイズを受け取り、条件に合った変数群を辞書形式で返す関数
# 引数：ベンチマーク名、問題サイズ
# 返り値：辞書（形式ー＞{"<変数名1>":<値1>, "<変数名2>":<値2>}）

def retDictAboutInitVars(benchmarkName, benchmarkClass):
    retDict = {}
    if benchmarkName=="cg":
        if benchmarkClass=="S":
            retDict["na"] = 1400
            retDict["nonzer"] = 7
            retDict["niter"] = 15
            retDict["shift"] = 24
        elif benchmarkClass=="W":
            retDict["na"] = 7000
            retDict["nonzer"] = 8
            retDict["niter"] = 15
            retDict["shift"] = 25
        elif benchmarkClass=="A":
            retDict["na"] = 14000
            retDict["nonzer"] = 11
            retDict["niter"] = 15
            retDict["shift"] = 20
        elif benchmarkClass=="B":
            retDict["na"] = 75000
            retDict["nonzer"] = 13
            retDict["niter"] = 75
            retDict["shift"] = 60
        elif benchmarkClass=="C":
            retDict["na"] = 150000
            retDict["nonzer"] = 15
            retDict["niter"] = 75
            retDict["shift"] = 110
        elif benchmarkClass=="D":
            retDict["na"] = 1500000
            retDict["nonzer"] = 21
            retDict["niter"] = 100
            retDict["shift"] = 500
        elif benchmarkClass=="E":
            retDict["na"] = 9000000
            retDict["nonzer"] = 26
            retDict["niter"] = 100
            retDict["shift"] = 1500
        elif benchmarkClass=="F":
            retDict["na"] = 54000000
            retDict["nonzer"] = 26
            retDict["niter"] = 100
            retDict["shift"] = 1500
    elif benchmarkName=="ep":
        if benchmarkClass=="S":
            retDict["m"] = 24
        elif benchmarkClass=="W":
            retDict["m"] = 25
        elif benchmarkClass=="A":
            retDict["m"] = 28
        elif benchmarkClass=="B":
            retDict["m"] = 30
        elif benchmarkClass=="C":
            retDict["m"] = 32
        elif benchmarkClass=="D":
            retDict["m"] = 36
        elif benchmarkClass=="E":
            retDict["m"] = 40
        elif benchmarkClass=="F":
            retDict["m"] = 44
    elif benchmarkName=="ft":
        if benchmarkClass=="S":
            retDict["nx"] = 64
            retDict["ny"] = 64
            retDict["nz"] = 64
            retDict["niter_default"] = 6
        elif benchmarkClass=="W":
            retDict["nx"] = 128
            retDict["ny"] = 128
            retDict["nz"] = 32
            retDict["niter_default"] = 6
        elif benchmarkClass=="A":
            retDict["nx"] = 256            
            retDict["ny"] = 256
            retDict["nz"] = 128
            retDict["niter_default"] = 6
        elif benchmarkClass=="B":
            retDict["nx"] = 512
            retDict["ny"] = 512
            retDict["nz"] = 256
            retDict["niter_default"] = 20
        elif benchmarkClass=="C":
            retDict["nx"] = 512
            retDict["ny"] = 512
            retDict["nz"] = 512
            retDict["niter_default"] = 20
        elif benchmarkClass=="D":
            retDict["nx"] = 2048
            retDict["ny"] = 1024
            retDict["nz"] = 1024
            retDict["niter_default"] = 25
        elif benchmarkClass=="E":
            retDict["nx"] = 4096
            retDict["ny"] = 2048
            retDict["nz"] = 2048
            retDict["niter_default"] = 25
        elif benchmarkClass=="F":
            retDict["nx"] = 8192
            retDict["ny"] = 4096
            retDict["nz"] = 4096
            retDict["niter_default"] = 25
    elif benchmarkName=="is":
        if benchmarkClass=="S":
            retDict["TOTAL_KEY_LOG_2"] = 16
            retDict["MAX_KEY_LOG_2"]   = 11
        elif benchmarkClass=="W":
            retDict["TOTAL_KEY_LOG_2"] = 20
            retDict["MAX_KEY_LOG_2"]   = 16
        elif benchmarkClass=="A":
            retDict["TOTAL_KEY_LOG_2"] = 23
            retDict["MAX_KEY_LOG_2"]   = 19
        elif benchmarkClass=="B":
            retDict["TOTAL_KEY_LOG_2"] = 25
            retDict["MAX_KEY_LOG_2"]   = 21
        elif benchmarkClass=="C":
            retDict["TOTAL_KEY_LOG_2"] = 27
            retDict["MAX_KEY_LOG_2"]   = 23
        elif benchmarkClass=="D":
            retDict["TOTAL_KEY_LOG_2"] = 31
            retDict["MAX_KEY_LOG_2"]   = 27
        elif benchmarkClass=="E":
            retDict["TOTAL_KEY_LOG_2"] = 35
            retDict["MAX_KEY_LOG_2"]   = 31
        elif benchmarkClass=="F":
            pass
    elif benchmarkName=="mg":
        if benchmarkClass=="S":
            retDict["nx_default"] = 32
            retDict["ny_default"] = 32
            retDict["nz_default"] = 32
            retDict["nit_default"] = 4
        elif benchmarkClass=="W":
            retDict["nx_default"] = 128
            retDict["ny_default"] = 128
            retDict["nz_default"] = 128
            retDict["nit_default"] = 4
        elif benchmarkClass=="A":
            retDict["nx_default"] = 256
            retDict["ny_default"] = 256
            retDict["nz_default"] = 256
            retDict["nit_default"] = 4
        elif benchmarkClass=="B":
            retDict["nx_default"] = 256
            retDict["ny_default"] = 256
            retDict["nz_default"] = 256
            retDict["nit_default"] = 20
        elif benchmarkClass=="C":
            retDict["nx_default"] = 512
            retDict["ny_default"] = 512
            retDict["nz_default"] = 512
            retDict["nit_default"] = 20
        elif benchmarkClass=="D":
            retDict["nx_default"] = 2048
            retDict["ny_default"] = 1024
            retDict["nz_default"] = 1024
            retDict["nit_default"] = 50
        elif benchmarkClass=="E":
            retDict["nx_default"] = 2048
            retDict["ny_default"] = 2048
            retDict["nz_default"] = 2048
            retDict["nit_default"] = 50
        elif benchmarkClass=="F":
            retDict["nx_default"] = 4096
            retDict["ny_default"] = 4096
            retDict["nz_default"] = 4096
            retDict["nit_default"] = 50
    elif benchmarkName=="lu":
        if benchmarkClass=="S":
            retDict["isiz01"]        = 12
            retDict["isiz02"]        = 12
            retDict["isiz03"]        = 12
            retDict["itmax_default"] = 50
            retDict["dt_default"]    = 0.5
        elif benchmarkClass=="W":
            retDict["isiz01"]        = 33
            retDict["isiz02"]        = 33
            retDict["isiz03"]        = 33
            retDict["itmax_default"] = 300
            retDict["dt_default"]    = 0.0015
        elif benchmarkClass=="A":
            retDict["isiz01"]        = 64
            retDict["isiz02"]        = 64
            retDict["isiz03"]        = 64
            retDict["itmax_default"] = 250
            retDict["dt_default"]    = 2
        elif benchmarkClass=="B":
            retDict["isiz01"]        = 102
            retDict["isiz02"]        = 102
            retDict["isiz03"]        = 102
            retDict["itmax_default"] = 250
            retDict["dt_default"]    = 2
        elif benchmarkClass=="C":
            retDict["isiz01"]        = 162
            retDict["isiz02"]        = 162
            retDict["isiz03"]        = 162
            retDict["itmax_default"] = 250
            retDict["dt_default"]    = 2
        elif benchmarkClass=="D":
            retDict["isiz01"]        = 408
            retDict["isiz02"]        = 408
            retDict["isiz03"]        = 408
            retDict["itmax_default"] = 300
            retDict["dt_default"]    = 1
        elif benchmarkClass=="E":
            retDict["isiz01"]        = 1020
            retDict["isiz02"]        = 1020
            retDict["isiz03"]        = 1020
            retDict["itmax_default"] = 300
            retDict["dt_default"]    = 0.5
        elif benchmarkClass=="F":
            retDict["isiz01"]        = 2560
            retDict["isiz02"]        = 2560
            retDict["isiz03"]        = 2560
            retDict["itmax_default"] = 300
            retDict["dt_default"]    = 0.2
    elif benchmarkName=="sp":
        if benchmarkClass=="S":
            retDict["problem_size"]  = 12
            retDict["niter_default"] = 100
            retDict["dt_default"]    = 0.015
        elif benchmarkClass=="W":
            retDict["problem_size"]  = 36
            retDict["niter_default"] = 400
            retDict["dt_default"]    = 0.0015
        elif benchmarkClass=="A":
            retDict["problem_size"]  = 64
            retDict["niter_default"] = 400
            retDict["dt_default"]    = 0.0015
        elif benchmarkClass=="B":
            retDict["problem_size"]  = 102
            retDict["niter_default"] = 400
            retDict["dt_default"]    = 0.001
        elif benchmarkClass=="C":
            retDict["problem_size"]  = 162
            retDict["niter_default"] = 400
            retDict["dt_default"]    = 0.00067
        elif benchmarkClass=="D":
            retDict["problem_size"]  = 408
            retDict["niter_default"] = 500
            retDict["dt_default"]    = 0.0003
        elif benchmarkClass=="E":
            retDict["problem_size"]  = 1020
            retDict["niter_default"] = 500
            retDict["dt_default"]    = 0.0001
        elif benchmarkClass=="F":
            retDict["problem_size"]  = 2560
            retDict["niter_default"] = 500
            retDict["dt_default"]    = 0.000015
    return retDict

def test_addInitDataToRawDF():
    # rawDF の作成（cg）
    functionName = [
        "functionAtCG",
    ]
    functionCallNum = [1]
    benchmarkName = ["cg"]
    benchmarkClass = ["A"]
    process = [2]
    rawDict = {
        "functionName": functionName,
        "functionCallNum": functionCallNum,
        "benchmarkName": benchmarkName,
        "benchmarkClass": benchmarkClass,
        "process": process,
    }
    rawDF = pd.DataFrame(rawDict)
    originalRawDF = rawDF.copy()
    # rawDF に手動で関数のやっていることを実施した DF を作成
    rawDF["na"] = 14000
    rawDF["nonzer"] = 11
    rawDF["niter"] = 15
    rawDF["shift"] = "20.d0"
    # 比較して正しいことを確認
    retDF = addInitDataToRawDF(originalRawDF)
    pd.testing.assert_frame_equal(retDF, rawDF)

    # rawDF の作成（ep）
    functionName = [
        "functionAtEP",
    ]
    functionCallNum = [2]
    benchmarkName = ["ep"]
    benchmarkClass = ["B"]
    process = [4]
    rawDict = {
        "functionName": functionName,
        "functionCallNum": functionCallNum,
        "benchmarkName": benchmarkName,
        "benchmarkClass": benchmarkClass,
        "process": process,
    }
    rawDF = pd.DataFrame(rawDict)
    originalRawDF = rawDF.copy()
    # rawDF に手動で関数のやっていることを実施した DF を作成
    rawDF["m"] = 30
    # 比較して正しいことを確認
    retDF = addInitDataToRawDF(originalRawDF)
    pd.testing.assert_frame_equal(retDF, rawDF)
    
    # rawDF の作成（ft）
    functionName = [
        "functionAtFT",
    ]
    functionCallNum = [4]
    benchmarkName = ["ft"]
    benchmarkClass = ["C"]
    process = [8]
    rawDict = {
        "functionName": functionName,
        "functionCallNum": functionCallNum,
        "benchmarkName": benchmarkName,
        "benchmarkClass": benchmarkClass,
        "process": process,
    }
    rawDF = pd.DataFrame(rawDict)
    originalRawDF = rawDF.copy()
    # rawDF に手動で関数のやっていることを実施した DF を作成
    rawDF["d1"] = 512
    rawDF["d2"] = 512
    rawDF["d3"] = 512
    rawDF["nt"] = 20
    # 比較して正しいことを確認
    retDF = addInitDataToRawDF(originalRawDF)
    pd.testing.assert_frame_equal(retDF, rawDF)
    
    # rawDF の作成（is）
    functionName = [
        "functionAtIS",
    ]
    functionCallNum = [8]
    benchmarkName = ["is"]
    benchmarkClass = ["D"]
    process = [16]
    rawDict = {
        "functionName": functionName,
        "functionCallNum": functionCallNum,
        "benchmarkName": benchmarkName,
        "benchmarkClass": benchmarkClass,
        "process": process,
    }
    rawDF = pd.DataFrame(rawDict)
    originalRawDF = rawDF.copy()
    # rawDF に手動で関数のやっていることを実施した DF を作成
    rawDF["TOTAL_KEYS_LOG_2"] = 31
    rawDF["MAX_KEY_LOG_2"] = 27
    # 比較して正しいことを確認
    retDF = addInitDataToRawDF(originalRawDF)
    pd.testing.assert_frame_equal(retDF, rawDF)
    
    # rawDF の作成（lu）
    functionName = [
        "functionAtLU",
    ]
    functionCallNum = [16]
    benchmarkName = ["lu"]
    benchmarkClass = ["E"]
    process = [32]
    rawDict = {
        "functionName": functionName,
        "functionCallNum": functionCallNum,
        "benchmarkName": benchmarkName,
        "benchmarkClass": benchmarkClass,
        "process": process,
    }
    rawDF = pd.DataFrame(rawDict)
    originalRawDF = rawDF.copy()
    # 比較して正しいことを確認
    retDF = addInitDataToRawDF(originalRawDF)
    pd.testing.assert_frame_equal(retDF, rawDF)
    # rawDF に手動で関数のやっていることを実施した DF を作成
    rawDF["isiz01"] = 1020
    rawDF["isiz02"] = 1020
    rawDF["isiz03"] = 1020
    rawDF["itmax_default"] = 300
    rawDF["itmax_default"] = 0.5

    # rawDF の作成（mg）
    functionName = [
        "functionAtMG",
    ]
    functionCallNum = [32]
    benchmarkName = ["mg"]
    benchmarkClass = ["F"]
    process = [64]
    rawDict = {
        "functionName": functionName,
        "functionCallNum": functionCallNum,
        "benchmarkName": benchmarkName,
        "benchmarkClass": benchmarkClass,
        "process": process,
    }
    rawDF = pd.DataFrame(rawDict)
    originalRawDF = rawDF.copy()
    # 比較して正しいことを確認
    retDF = addInitDataToRawDF(originalRawDF)
    pd.testing.assert_frame_equal(retDF, rawDF)
    # rawDF に手動で関数のやっていることを実施した DF を作成
    rawDF["nx_default"] = 8192
    rawDF["ny_default"] = 4096
    rawDF["nz_default"] = 4096
    rawDF["nit_default"] = 50
    
    pass
    return rawDF


sampleRawDF = returnCollectedExistingData(
    benchmarkNames=["cg"],
    classes=["A", "B", "C", "D", "E", "F"],
    processes=[2, 4, 8, 16, 32, 64, 128, 256],
    csvDirPath=csvDirPath,
)
sampleRawDF

test_addInitDataToRawDF()

   functionName  functionCallNum benchmarkName benchmarkClass  process
0  functionAtCG                1            cg              A        2
benchmarkName=cg, benchmarkClass=A
   functionName  functionCallNum benchmarkName benchmarkClass  process
0  functionAtCG                1            cg              A        2


AssertionError: DataFrame are different

DataFrame shape mismatch
[left]:  (1, 5)
[right]: (1, 9)